In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup  #HTML 파싱
from urllib.parse import quote  #검색어 인코딩
import time  #슬립 사용
import requests  #HTML 가져오기
import datetime

In [2]:
jan_contents = pd.read_csv('./LG_data/00. 원본/contents_20240101.csv')
feb_contents = pd.read_csv('./LG_data/00. 원본/contents_20240201.csv')
mar_contents = pd.read_csv('./LG_data/00. 원본/contents_20240301.csv')

jan_contents.info()
feb_contents.info()
mar_contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24088 entries, 0 to 24087
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           24088 non-null  int64 
 1   series_nm       7526 non-null   object
 2   super_asset_nm  24088 non-null  object
 3   ct_cl           24088 non-null  object
 4   genre_of_ct_cl  24088 non-null  object
 5   SMRY            24088 non-null  object
 6   ACTR_DISP       24088 non-null  object
 7   disp_rtm        22509 non-null  object
 8   log_dt          24088 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24244 entries, 0 to 24243
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           24244 non-null  int64 
 1   series_nm       8477 non-null   object
 2   super_asset_nm  24244 non-null  object
 3   ct_cl           24244 non

In [3]:
# jan_to_mar_contents = pd.concat([jan_contents, feb_contents, mar_contents])
# jan_to_mar_contents.info() #22557
# concat : index가 중복되는 문제

# 1) df. reset_index(drop=True, inplace = True)
# : reset_index 특성상 기존의 인덱스는 feature로 생성되고
# 새로운 인덱스가 sort되어서 할당 > drop = True 옵션 작성 필요

# 2) df = pd.concat([df1, df2], ignore_index = True)

# Sol---------------------------------------------------------------
#1) 
# jan_to_mar_contents.reset_index(drop = True, inplace = True)
# jan_to_mar_contents

#2) 
jan_to_mar_contents = pd.concat([jan_contents, feb_contents, mar_contents], ignore_index= True)
jan_to_mar_contents


,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,66056000,NaN,유혹2,영화,에로틱,"홍콩의 핫한 클럽 하이X의 사장이자, 밤의 황태자 호궈펑. 사랑 빼곤 모든 것을 다...",하호문,NaN,20240104140053
1,66056000,NaN,2차노출,영화,멜로,성형외과 전문의 송치는 어느 날 자신의 애인 류동과 가장 친한 친구 샤오시가 서로를...,"판빙빙,풍소봉,곽사연,요안렴",NaN,20240112024534
2,61779000,NaN,마들렌,영화,멜로,소설가를 꿈꾸는 지석. 헤어 디자이너를 꿈꾸는 희진. 몇 번의 우연한 만남이 이어지...,"조인성,신민아,박정아,강래연,김수로,최규환",01:56,20240124221213
3,67170000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52,20240105090808
4,66547000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52,20240106013407
...,...,...,...,...,...,...,...,...,...
73242,60784000,매화홍도 : 미스터 앤 미세스 첸,매화홍도 : 미스터 앤 미세스 첸 24회,TV드라마,외화 시리즈,진가평과 국민당 요원이 합작하여 계획한 위조지폐 공장 폭파 계획은 일본인에게 작전을...,"관효동,한동군,전뢰,장의",00:45,20240313001030
73243,64659000,눈떠보니 라떼,눈떠보니 라떼 3회,TV드라마,외화 시리즈,본 회차는 방송사의 사정으로 줄거리를 제공하지 않습니다.,-,00:30,20240325095701
73244,66056000,눈떠보니 라떼,눈떠보니 라떼 3회,TV드라마,외화 시리즈,본 회차는 방송사의 사정으로 줄거리를 제공하지 않습니다.,-,00:30,20240323120714
73245,66056000,내 이야기는 길어,내 이야기는 길어 1회,TV드라마,외화 시리즈,스키야키와 자전거 외. 오늘도 하는 일 없이 엄마에게 빌붙어 사는 백수 키시베 미츠...,"이쿠타 토마,야스다 켄,코이케 에이코,키요하라 카야",00:46,20240329001010


제목 끝에 . 제거

In [6]:
for idx in jan_to_mar_contents.index :
  jan_to_mar_contents.loc[idx, 'super_asset_nm'] = re.sub('[.]', '', jan_to_mar_contents.loc[idx, 'super_asset_nm'])

jan_to_mar_contents.head()

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,66056000,NaN,유혹2,영화,에로틱,"홍콩의 핫한 클럽 하이X의 사장이자, 밤의 황태자 호궈펑. 사랑 빼곤 모든 것을 다...",하호문,NaN,20240104140053
1,66056000,NaN,2차노출,영화,멜로,성형외과 전문의 송치는 어느 날 자신의 애인 류동과 가장 친한 친구 샤오시가 서로를...,"판빙빙,풍소봉,곽사연,요안렴",NaN,20240112024534
2,61779000,NaN,마들렌,영화,멜로,소설가를 꿈꾸는 지석. 헤어 디자이너를 꿈꾸는 희진. 몇 번의 우연한 만남이 이어지...,"조인성,신민아,박정아,강래연,김수로,최규환",01:56,20240124221213
3,67170000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52,20240105090808
4,66547000,NaN,1번가의 기적,영화,코미디,"재개발의 막중한 임무를 띠고 에쿠스를 끌며 폼 나게 1번가에 나타난 날건달, 필제....","임창정,하지원,주현,정두홍,이훈,강예원,박창익,박유선",01:52,20240106013407


In [7]:
jan_to_mar_contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73247 entries, 0 to 73246
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           73247 non-null  int64 
 1   series_nm       24361 non-null  object
 2   super_asset_nm  73247 non-null  object
 3   ct_cl           73247 non-null  object
 4   genre_of_ct_cl  73247 non-null  object
 5   SMRY            73247 non-null  object
 6   ACTR_DISP       73247 non-null  object
 7   disp_rtm        68376 non-null  object
 8   log_dt          73247 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 5.0+ MB


datetime 형 변환 > 제목 끝 . 제거

In [8]:
jan_to_mar_contents.to_csv(path_or_buf = './LG_data/01. 전처리/contents_전처리_19.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

In [9]:
contents_movie = jan_to_mar_contents[jan_to_mar_contents['ct_cl'] == '영화']
contents_movie.info() #47883

contents_series = jan_to_mar_contents[jan_to_mar_contents['ct_cl'] != '영화']
contents_series.info() #25364

<class 'pandas.core.frame.DataFrame'>
Index: 47883 entries, 0 to 73241
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           47883 non-null  int64 
 1   series_nm       24 non-null     object
 2   super_asset_nm  47883 non-null  object
 3   ct_cl           47883 non-null  object
 4   genre_of_ct_cl  47883 non-null  object
 5   SMRY            47883 non-null  object
 6   ACTR_DISP       47883 non-null  object
 7   disp_rtm        43012 non-null  object
 8   log_dt          47883 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 3.7+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 25364 entries, 21 to 73246
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           25364 non-null  int64 
 1   series_nm       24337 non-null  object
 2   super_asset_nm  25364 non-null  object
 3   ct_cl           25364 non-null  ob

In [10]:
contents_movie.to_csv(path_or_buf = './LG_data/01. 전처리/contents_movie_전처리_19.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')

contents_series.to_csv(path_or_buf = './LG_data/01. 전처리/contents_series_전처리_19.csv', sep = ',', header= True,
                    mode= 'a', encoding= 'cp949')   